In [ ]:
pip install requests
pip install pandas

In [2]:
import requests
import pandas as pd

In [3]:
BASE_URL = "https://api.openaq.org/v3"

headers = {'X-API-Key': '4417e9a6bba30a0bff7af8ec79240fcb19aded4cfdf587f1d48c0fbfbff883b3'}

resp = requests.get(f"https://api.openaq.org/v3/locations?iso=VN", headers=headers, timeout=30)
resp.raise_for_status()
data = resp.json()


In [ ]:
data

In [11]:
rows = []

for loc in data.get("results", []):
    location_id = loc.get("id")
    location_name = loc.get("name")
    city = loc.get("locality")
    timezone = loc.get("timezone")

    country = loc.get("country", {})
    country_code = country.get("code")

    coords = loc.get("coordinates", {})
    latitude = coords.get("latitude")
    longitude = coords.get("longitude")

    dt_first = loc.get("datetimeFirst", {})
    dt_last = loc.get("datetimeLast", {})

    datetime_first = dt_first.get("utc") if isinstance(dt_first, dict) else None
    datetime_last = dt_last.get("utc") if isinstance(dt_last, dict) else None

    sensors = loc.get("sensors", [])

    for sensor in sensors:
        param = sensor.get("parameter", {})

        rows.append({
            "location_id": location_id,
            "location_name": location_name,
            "city": city,
            "country_code": country_code,
            "timezone": timezone,
            "latitude": latitude,
            "longitude": longitude,
            "sensor_id": sensor.get("id"),
            "parameter": param.get("name"),
            "unit": param.get("units"),
            "datetime_first": datetime_first,
            "datetime_last": datetime_last
        })

df_raw = pd.DataFrame(rows)


In [12]:
df_raw

,location_id,location_name,city,country_code,timezone,latitude,longitude,sensor_id,parameter,unit,datetime_first,datetime_last
0,18,SPARTAN - Vietnam Acad. Sci.,None,VN,Asia/Bangkok,21.048000,105.800000,37,pm25,µg/m³,None,None
1,2446,US Diplomatic Post: Ho Chi Minh City,None,VN,Asia/Ho_Chi_Minh,10.783041,106.700722,4681,pm25,µg/m³,2016-02-29T01:00:00Z,2016-11-09T16:00:00Z
2,2539,US Diplomatic Post: Hanoi,None,VN,Asia/Bangkok,21.021770,105.819002,5049,pm25,µg/m³,2016-01-30T01:00:00Z,2016-11-09T16:00:00Z
3,7440,US Diplomatic Post: Ho Chi Minh City,Ho Chi Minh City,VN,Asia/Ho_Chi_Minh,10.782773,106.700035,21631,pm25,µg/m³,2016-11-09T17:00:00Z,2025-03-04T12:00:00Z
4,7441,Hanoi,N/A,VN,Asia/Bangkok,21.021939,105.818806,21632,pm25,µg/m³,2016-11-09T18:00:00Z,2025-04-09T15:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...
186,6123215,OceanPark,None,VN,Asia/Bangkok,20.993300,105.944100,14581374,pm1,µg/m³,2025-11-08T06:00:00Z,2025-12-22T02:00:00Z
187,6123215,OceanPark,None,VN,Asia/Bangkok,20.993300,105.944100,14581375,pm25,µg/m³,2025-11-08T06:00:00Z,2025-12-22T02:00:00Z
188,6123215,OceanPark,None,VN,Asia/Bangkok,20.993300,105.944100,14581376,relativehumidity,%,2025-11-08T06:00:00Z,2025-12-22T02:00:00Z
189,6123215,OceanPark,None,VN,Asia/Bangkok,20.993300,105.944100,14581377,temperature,c,2025-11-08T06:00:00Z,2025-12-22T02:00:00Z


In [14]:
df_raw["datetime_last"] = pd.to_datetime(df_raw["datetime_last"], errors="coerce")


In [17]:
from datetime import datetime, timedelta

lookback_days = 30
import pytz

cutoff_date = (datetime.utcnow() - timedelta(days=lookback_days)).replace(tzinfo=pytz.UTC)

df_active = df_raw[
    df_raw["datetime_last"].notna() &
    (df_raw["datetime_last"] >= cutoff_date)
]


C:\Users\cau.tran\AppData\Local\Temp\ipykernel_16320\3950650961.py:6: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  cutoff_date = (datetime.utcnow() - timedelta(days=lookback_days)).replace(tzinfo=pytz.UTC)


In [6]:
params = {
    "datetime_from": "2025-04-01T00:00:00Z",
    "datetime_to": "2025-04-02T00:00:00Z",
    "limit": 100
}

In [7]:
sensor_id = int(df['sensor_id'].iloc[0])  # lấy sensor đầu tiên

resp = requests.get(
    f"{BASE_URL}/sensors/{sensor_id}/measurements",
    headers=headers,
    params=params,
    timeout=30
)

resp.raise_for_status()
sensor = resp.json()
print(sensor)


{'meta': {'name': 'openaq-api', 'website': '/', 'page': 1, 'limit': 100, 'found': 0}, 'results': []}


In [ ]:
all_measurements = []

for sensor_id in df['sensor_id'].unique():
    print(f"Fetching sensor {sensor_id}")

    resp = requests.get(
        f"{BASE_URL}/sensors/{sensor_id}/measurements",
        headers=headers,
        params=params,
        timeout=30
    )

    if resp.status_code != 200:
        print(f"Failed sensor {sensor_id}")
        continue

    data = resp.json()

    for m in data.get("results", []):
        period = m.get("period", {})
        dt = period.get("datetimeFrom", {})

        all_measurements.append({
            "sensor_id": sensor_id,
            "datetime": dt.get("utc"),
            "value": m.get("value"),
            "parameter": m.get("parameter", {}).get("name"),
            "unit": m.get("parameter", {}).get("units")
        })

Fetching sensor 37
Fetching sensor 4681
Fetching sensor 5049
Fetching sensor 21631
Fetching sensor 21632
Fetching sensor 1552804
Fetching sensor 1552809
Fetching sensor 1553310
Fetching sensor 1553316
Fetching sensor 1553318
Fetching sensor 1742521
Fetching sensor 6566240
Fetching sensor 7771981
Fetching sensor 7772033
Fetching sensor 7772095
Fetching sensor 7772012
Fetching sensor 7772103
Fetching sensor 7771974
Fetching sensor 7772105
Fetching sensor 7772023
Fetching sensor 7772024
Fetching sensor 7888636
Fetching sensor 7772022
Fetching sensor 7772048
Fetching sensor 7772032
Fetching sensor 7771973
Fetching sensor 7772064
Fetching sensor 7772027
Fetching sensor 7772077
Fetching sensor 7772046
Fetching sensor 7772083
Fetching sensor 7771978
Fetching sensor 7772082
Fetching sensor 7772104
Fetching sensor 7772081
Fetching sensor 7772045
Fetching sensor 7771991
Fetching sensor 7771992
Fetching sensor 7771983
Fetching sensor 7772030
Fetching sensor 7772101
Fetching sensor 7772040
Fetchin

In [ ]:
import json

with open('sample_raw.json', 'r') as f:
    data = json.load(f)

print('='*80)
print('KIỂM TRA CẤU TRÚC VÀ CÁC CHỈ SỐ TRONG sample_raw.json')
print('='*80)

if 'meta' in data:
    meta = data['meta']
    print('\n[THÔNG TIN METADATA]:')
    print('  • API Name:', meta.get('name'))
    print('  • Total Records Found:', meta.get('found'))
    print('  • Extracted At:', meta.get('extracted_at'))

if 'results' in data:
    results = data['results']
    print('\n[DỮ LIỆU - RESULTS]:')
    print('  • Tổng số records:', len(results))
    
    if len(results) > 0:
        first_record = results[0]
        fields = list(first_record.keys())
        print('  • Số fields mỗi record:', len(fields))
        print('\n[CÁC FIELDS CÓ TRONG DỮ LIỆU]:')
        for i, field in enumerate(fields, 1):
            value = first_record[field]
            print('    %d. %-20s = %s' % (i, field, value))
        
        # Get all unique parameters
        parameters = set()
        for record in results:
            param = record.get('parameter')
            if param:
                parameters.add(param)
        
        print('\n[CÁC CHỈ SỐ/PARAMETERS CÓ TRONG FILE]:')
        print('  • Tổng số unique parameters:', len(parameters))
        for param in sorted(parameters):
            count = sum(1 for r in results if r.get('parameter') == param)
            print('    ✓ %-20s (xuất hiện %d lần)' % (param, count))
        
        # Check for null values
        print('\n[KIỂM TRA CHẤT LƯỢNG DỮ LIỆU]:')
        null_count = sum(1 for r in results if r.get('value') is None)
        valid_count = len(results) - null_count
        print('  • Records có value:', valid_count, '/', len(results))
        if valid_count == len(results):
            print('  ✅ TẤT CẢ RECORDS ĐỀU CÓ DỮ LIỆU')
        else:
            print('  ⚠️  CÓ', null_count, 'RECORDS KHÔNG CÓ GIÁ TRỊ')


[INFO] Analyzing sample_raw.json structure
[INFO] Total records in results: 574

[INFO] Metadata from file:
  name: openaq-api
  website: https://api.openaq.org/v3
  found: 574
  extracted_at: 2025-12-28T06:17:56.789373

[INFO] Sample record structure:
  Keys in first record: ['sensor_id', 'datetime', 'value', 'parameter', 'unit', 'extracted_at', 'location_id', 'location_name', 'city', 'timezone', 'country', 'latitude', 'longitude']

[INFO] First record content:
  sensor_id: 7772048
  datetime: 2025-12-26 23:00:00+00:00
  value: 141.51
  parameter: pm10
  unit: µg/m³
  extracted_at: 2025-12-28 06:17:56.773792
  location_id: 2161292
  location_name: Số 46, phố Lưu Quang Vũ
  city: Hanoi
  timezone: Asia/Bangkok
  country: VN
  latitude: 21.0152
  longitude: 105.7999

[INFO] Data types in sample record:
  sensor_id: int
  datetime: str
  value: float
  parameter: str
  unit: str
  extracted_at: str
  location_id: int
  location_name: str
  city: str
  timezone: str
  country: str
  latit

NameError: name 'pd' is not defined